In [ ]:
#Look Ma a Spike


In [1]:
import math
import mne
from MEG_Tools import MEG
import matplotlib as plt
import numpy as np
plt.use('Qt5Agg')
meg = MEG('case_2225_with_spike_dipoles_sleep_2.mat')
mne_fif = meg.get_mne()

Creating RawArray with float64 data, n_channels=373, n_times=509000
    Range : 0 ... 508999 =      0.000 ...   508.999 secs
Ready.
Writing /Users/williamwilson/Library/CloudStorage/Dropbox/Ganz/MEG/MEG-Tools/case_2225_with_spike_dipoles_sleep_2.mat.tmp.raw.fif
Closing /Users/williamwilson/Library/CloudStorage/Dropbox/Ganz/MEG/MEG-Tools/case_2225_with_spike_dipoles_sleep_2.mat.tmp.raw.fif
[done]
Opening raw data file case_2225_with_spike_dipoles_sleep_2.mat.tmp.raw.fif...
Isotrak not found
    Range : 0 ... 508999 =      0.000 ...   508.999 secs
Ready.


In [ ]:
#First we find the location of a known spike

In [2]:
known_spike_time = meg.spikes[0]['begin']
known_spike_time = int(math.floor(known_spike_time))#round to second below
known_spike_time

51

In [3]:
#Make 1 second epochs
sfreq = mne_fif.info['sfreq']
events = np.array([[i, 0, 1] for i in range(0, mne_fif.n_times - int(sfreq), int(sfreq))], dtype=int)
epochs = mne.Epochs(mne_fif, events, event_id={'one_sec': 1}, tmin=0, tmax=1, baseline=None, preload=True)

Not setting metadata
508 matching events found
No baseline correction applied
0 projection items activated
Loading data for 508 events and 1001 original time points ...
0 bad epochs dropped


In [4]:
#Let's take a look....
first_spike = epochs[known_spike_time]

# Plot the first epoch
first_spike.plot()

Using matplotlib as 2D backend.


<MNEBrowseFigure size 1600x1600 with 4 Axes>

Dropped 0 epochs: 
The following epochs were marked as bad and are dropped:
[]
Channels marked as bad:
none


In [5]:
frequencies = np.arange(4, 70, 1)
power = mne.time_frequency.tfr_morlet(
    epochs[known_spike_time-1:known_spike_time+1], n_cycles=2, return_itc=False, freqs=frequencies, decim=3
)
power.plot(["MEG0242"])

[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.1s
[Parallel(n_jobs=1)]: Done  71 tasks      | elapsed:    0.5s
[Parallel(n_jobs=1)]: Done 161 tasks      | elapsed:    1.2s
[Parallel(n_jobs=1)]: Done 287 tasks      | elapsed:    2.2s


No baseline correction applied


[<Figure size 1280x960 with 2 Axes>]

In [6]:
frequencies = np.arange(4, 70, 1)
power = mne.time_frequency.tfr_morlet(
    epochs[12:14], n_cycles=2, return_itc=False, freqs=frequencies, decim=3
)
power.plot(["MEG0242"])

[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.1s
[Parallel(n_jobs=1)]: Done  71 tasks      | elapsed:    0.6s
[Parallel(n_jobs=1)]: Done 161 tasks      | elapsed:    1.2s
[Parallel(n_jobs=1)]: Done 287 tasks      | elapsed:    2.2s


No baseline correction applied


[<Figure size 1280x960 with 2 Axes>]

In [7]:
epochs[known_spike_time-1:known_spike_time+1].plot_image(picks=["MEG0242"])

Not setting metadata
2 matching events found
No baseline correction applied
0 projection items activated


[<Figure size 1280x960 with 4 Axes>]

In [8]:
epochlength = len(epochs)
# Creating spiked_list from meg.spikes
spiked_list = []
for n in range(len(meg.spikes)):
    begin = meg.spikes[n]['begin']
    spiked_list.append(int(math.floor(begin)))

# Remove duplicates and sort spiked_list
spiked_list = sorted(list(set(spiked_list)))

# Creating unspiked_list
unspiked_list = [i for i in range(epochlength) if i not in spiked_list]

spiked_epochs = epochs[spiked_list]
unspiked_epochs = epochs[unspiked_list]





In [9]:
spiked_evoked = spiked_epochs.average()
unspiked_evoked = unspiked_epochs.average()

mne.viz.plot_compare_evokeds(
    dict(spiked=spiked_evoked, unspiked=unspiked_evoked),
    legend="upper left",
    show_sensors="upper right",
)

Multiple channel types selected, returning one figure per type.
combining channels using "gfp"
combining channels using "gfp"
combining channels using "gfp"
combining channels using "gfp"
combining channels using "gfp"
combining channels using "gfp"


[<Figure size 1600x1200 with 2 Axes>,
 <Figure size 1600x1200 with 2 Axes>,
 <Figure size 1600x1200 with 2 Axes>]

In [10]:
evoked_diff = mne.combine_evoked([spiked_evoked, unspiked_evoked], weights=[1, -1])
evoked_diff.pick(picks="mag").plot_topo(color="r", legend=False)

<Figure size 1280x960 with 1 Axes>

In [11]:
#Hmmmm
spiked_evoked.plot_joint(picks="mag")
spiked_evoked.plot_topomap(times=[0.0, 0.08, 0.1, 0.12, 0.2], ch_type="mag")

No projector specified for this dataset. Please consider the method self.add_proj.


<MNEFigure size 1500x440 with 6 Axes>